In [2]:
import qiskit_nature
import qiskit
import pyscf

print("Qiskit version:", qiskit.__version__)
print("Qiskit Nature version:", qiskit_nature.__version__)
print("PySCF version:", pyscf.__version__)


Qiskit version: 1.4.2
Qiskit Nature version: 0.7.2
PySCF version: 2.8.0


In [ ]:
#H2 CCSD energy


from pyscf import gto, scf, cc

mol = gto.M(
    atom='H 0 0 0; H 0 0 0.735',
    basis='sto-3g',
    unit='Angstrom',
    charge=0,
    spin=0,
)

mf = scf.RHF(mol).run()
mycc = cc.CCSD(mf).run()

print("H2 PySCF CCSD Energy:", mycc.e_tot)


In [ ]:
#vqe for h2 no tapering no freezecore using uccsd


#H2 VQE + freeze core + qubit tapering (added transpilation & parameter assignment ;to remove error "unknown instruction")
#TRYING TO CHANGE EFFICIENTSU2 TO UCCSSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.quantum_info import Z2Symmetries
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_aer.primitives import EstimatorV2
from qiskit.providers.basic_provider import BasicSimulator
from scipy.optimize import minimize
import numpy as np
from qiskit.circuit.library import EfficientSU2

# Set up the molecule H2
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)], charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

# Apply freeze core transformer
print("Applying freeze core approximation...")
# fc_transformer = FreezeCoreTransformer()
# transformed_problem = fc_transformer.transform(problem)

# Get the transformed Hamiltonian
hamiltonian = problem.hamiltonian.second_q_op()
mapper = JordanWignerMapper()

# tapered_mapper = transformed_problem.get_tapered_mapper(mapper)
qubit_op = mapper.map(hamiltonian)


# Setup for VQE
backend = BasicSimulator()

# Creating ansatz circuit for the tapered Hamiltonian
num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals
hf_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, mapper, initial_state=hf_state)


# Create initial parameters with the correct shape
# First bind the circuit parameters to see what parameters we need
print(f"Circuit has {ansatz.num_parameters} parameters")
print(f"Parameter names: {ansatz.parameters}")

# Create initial parameters as a flat array with the correct length
initial_params = np.zeros(ansatz.num_parameters)
# initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)

# Setup the estimator and cost function
estimator = EstimatorV2()
cost_history = []

def cost_func(params):
    bound_circ = ansatz.assign_parameters(params)
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled_ansatz = pm.run(bound_circ)
    pub = (transpiled_ansatz, [qubit_op], [np.array([])])
    result = estimator.run([pub]).result()
    energy = result[0].data.evs[0]
    cost_history.append(energy)
    print(f"  Iteration {len(cost_history)} -> Energy: {energy:.6f}")
    return energy

# Run the VQE optimization
print("\nRunning VQE optimization for H2 with freeze core and qubit tapering...")
result = minimize(cost_func, initial_params, method="COBYLA")
nuclearenergy=problem.nuclear_repulsion_energy  
print(f"Nuclear energy: { nuclearenergy:.6f} Hartree")
print(f"Total energy: {result.fun + nuclearenergy:.6f} Hartree")
# Print final results
print("\nOptimization complete!")
print(f"Ground state energy of H2: {result.fun :.6f} Hartree")
print(f"Optimization success: {result.success}")
print(f"Number of iterations: {len(cost_history)}")



In [ ]:
#H2 VQE + freeze core + qubit tapering (added transpilation & parameter assignment ;to remove error "unknown instruction")
#TRYING TO CHANGE EFFICIENTSU2 TO UCCSSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.quantum_info import Z2Symmetries
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_aer.primitives import EstimatorV2
from qiskit.providers.basic_provider import BasicSimulator
from scipy.optimize import minimize
import numpy as np
from qiskit.circuit.library import EfficientSU2

# Set up the molecule H2
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)], charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

# Apply freeze core transformer
print("Applying freeze core approximation...")
fc_transformer = FreezeCoreTransformer()
transformed_problem = fc_transformer.transform(problem)

# Get the transformed Hamiltonian
hamiltonian = transformed_problem.second_q_ops()[0]
mapper = JordanWignerMapper()

tapered_mapper = transformed_problem.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)


# Setup for VQE
backend = BasicSimulator()

# Creating ansatz circuit for the tapered Hamiltonian
num_particles = transformed_problem.num_particles
num_spatial_orbitals = transformed_problem.num_spatial_orbitals
hf_state = HartreeFock(num_spatial_orbitals, num_particles, tapered_mapper)
ansatz = UCCSD(num_spatial_orbitals, num_particles, tapered_mapper, initial_state=hf_state)


# Create initial parameters with the correct shape
# First bind the circuit parameters to see what parameters we need
print(f"Circuit has {ansatz.num_parameters} parameters")
print(f"Parameter names: {ansatz.parameters}")

# Create initial parameters as a flat array with the correct length
initial_params = np.zeros(ansatz.num_parameters)
# initial_params = np.random.uniform(-0.1, 0.1, ansatz.num_parameters)

# Setup the estimator and cost function
estimator = EstimatorV2()
cost_history = []

def cost_func(params):
    bound_circ = ansatz.assign_parameters(params)
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled_ansatz = pm.run(bound_circ)
    pub = (transpiled_ansatz, [qubit_op], [np.array([])])
    result = estimator.run([pub]).result()
    energy = result[0].data.evs[0]
    cost_history.append(energy)
    print(f"  Iteration {len(cost_history)} -> Energy: {energy:.6f}")
    return energy

# Run the VQE optimization
print("\nRunning VQE optimization for H2 with freeze core and qubit tapering...")
result = minimize(cost_func, initial_params, method="COBYLA")
nuclearenergy=problem.nuclear_repulsion_energy  
frozen_core_energy = transformed_problem.hamiltonian.constants.get("FreezeCoreTransformer", 0.0)
print(f"Frozen core energy from hamiltonian constants: {frozen_core_energy} Hartree")
print(f"Nuclear energy: { nuclearenergy:.6f} Hartree")
print(f"Total energy: {result.fun + nuclearenergy:.6f} Hartree")
# Print final results
print("\nOptimization complete!")
print(f"Ground state energy of H2: {result.fun :.6f} Hartree")
print(f"Optimization success: {result.success}")
print(f"Number of iterations: {len(cost_history)}")